In [26]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

project_root = Path("/Users/david/Code/msc/project")
data_path = project_root / "data"
src_path = project_root / "src"
print(f"Looking for src at: {src_path}")
print(f"Does src directory exist: {src_path.exists()}")
if src_path.exists():
    print(f"Contents of src: {list(src_path.glob('*'))}")

sys.path.append(str(src_path))
print(f"Added to sys.path: {src_path}")
print(f"Current working directory: {Path.cwd()}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Looking for src at: /Users/david/Code/msc/project/src
Does src directory exist: True
Contents of src: [PosixPath('/Users/david/Code/msc/project/src/__pycache__'), PosixPath('/Users/david/Code/msc/project/src/utils.py'), PosixPath('/Users/david/Code/msc/project/src/.ipynb_checkpoints'), PosixPath('/Users/david/Code/msc/project/src/process.ipynb')]
Added to sys.path: /Users/david/Code/msc/project/src
Current working directory: /Users/david/Code/msc


In [ ]:
ZTF_CSV_PATH = data_path / "ztf.csv"
df = pd.read_csv(ZTF_CSV_PATH)
print(f"Loaded {len(df)} objects from ztf.csv (read-only)")
df.head()

Loaded 7070 objects from ztf.csv


,ZTFID,IAUID,RA,Dec,peakt,peakfilt,peakmag,peakabs,duration,rise,fade,type,redshift,b,A_V
0,ZTF17aabtvsy,SN2022yei,10:35:32.09,+37:38:59.0,1870.99,r,18.0303,-19.41,>34.229,>6.01,28.219,SN Ia,0.06922,59.641962,0.053
1,ZTF17aabvong,SN2024xxq,02:05:07.68,+11:14:55.1,2606.75,g,16.8039,-19.57,23.222,8.464,14.758,SN Ia,0.034,-47.664064,0.446
2,ZTF17aacldgo,SN2022zxv,03:09:24.36,-04:53:39.2,1897.75,g,18.7979,-18.91,>1077,>3.85,>1073.15,SN Ia,0.072,-50.332472,0.183
3,ZTF17aadlxmv,SN2020adv,08:29:47.59,+33:54:22.8,879.69,g,17.9475,-19.34,25.146,10.951,14.195,SN Ia,0.062,34.174702,0.106
4,ZTF18aaaibml,SN2020buc,10:08:31.27,+09:14:23.0,909.80,r,18.2606,-17.70,69.73,27.302,42.428,SN II,0.0348,47.637802,0.092


In [ ]:
# User input prompts
def get_user_input():
    """Get ID (or 'all' for all rows) and --force flag from user input."""
    ztf_id = input("Enter ZTF ID (e.g., ZTF17aabtvsy) or 'all' for all rows: ").strip()
    force = input("Force run? (y/n, default: n): ").strip().lower() == 'y'
    return ztf_id, force

# Get user inputs
ztf_id, force_run = get_user_input()
process_all = ztf_id.lower() in ['all', 'a', '']

if process_all:
    print(f"\nProcessing ALL {len(df)} objects from ztf.csv")
    print(f"Force run: {force_run}")
    # Create a copy to ensure original df is never modified
    objects_to_process = df.copy()
else:
    print(f"\nProcessing ZTF ID: {ztf_id}")
    print(f"Force run: {force_run}")
    objects_to_process = df[df['ZTFID'] == ztf_id].copy()
    
    if objects_to_process.empty:
        print(f"Warning: No data found for ZTF ID {ztf_id}")
        print(f"Available IDs (first 10): {df['ZTFID'].head(10).tolist()}")
    else:
        print(f"\nFound {len(objects_to_process)} entry(ies) for {ztf_id}")
        print(objects_to_process[['ZTFID', 'IAUID', 'type', 'peakmag', 'peakt']].to_string())


Processing ALL 7070 objects from ztf.csv
Force run: False


In [31]:
# Process objects
if not objects_to_process.empty:
    total = len(objects_to_process)
    
    for idx, (_, obj_row) in enumerate(objects_to_process.iterrows(), 1):
        ztf_id = obj_row['ZTFID']
        print(f"[{idx}/{total}] {ztf_id}: Success")
    
    print(f"\n✓ Processing complete for {total} object(s)")
else:
    print("\n✗ Cannot proceed without valid ZTF ID")

[1/7070] ZTF17aabtvsy: Success
[2/7070] ZTF17aabvong: Success
[3/7070] ZTF17aacldgo: Success
[4/7070] ZTF17aadlxmv: Success
[5/7070] ZTF18aaaibml: Success
[6/7070] ZTF18aaaonon: Success
[7/7070] ZTF18aaaooqj: Success
[8/7070] ZTF18aaaqexr: Success
[9/7070] ZTF18aacemcn: Success
[10/7070] ZTF18aacnlxz: Success
[11/7070] ZTF18aadlaxo: Success
[12/7070] ZTF18aadmssd: Success
[13/7070] ZTF18aadsuxd: Success
[14/7070] ZTF18aadxnul: Success
[15/7070] ZTF18aadzfso: Success
[16/7070] ZTF18aaeqjmc: Success
[17/7070] ZTF18aaermez: Success
[18/7070] ZTF18aaewdgq: Success
[19/7070] ZTF18aafdigb: Success
[20/7070] ZTF18aafzers: Success
[21/7070] ZTF18aagkwgz: Success
[22/7070] ZTF18aagtwyh: Success
[23/7070] ZTF18aahatvc: Success
[24/7070] ZTF18aahfbqp: Success
[25/7070] ZTF18aahtjsc: Success
[26/7070] ZTF18aahvndq: Success
[27/7070] ZTF18aailmnv: Success
[28/7070] ZTF18aaisaqo: Success
[29/7070] ZTF18aaisqmw: Success
[30/7070] ZTF18aaiwzie: Success
[31/7070] ZTF18aaiykoz: Success
[32/7070] ZTF18aa